In [0]:
import os
import numpy as np
import pandas as pd

In [0]:
def get_At_matrix(data, data_names, t, n = 24, perc=10):
  # Arrays with n samples for t moment of time 
  data_t = []
  for i in range(len(data_names)):
    data_t += [vol_data[:,i][int(t-n):t]]
  data_t = np.array(data_t)
  # List to collect names without _hourly.csv
  crypto_names = []
  # List to collect medians during the iteration over the array of volume sets
  medians = []
  for i in range(len(data_names)):
    med = np.median(data_t[i])
    crypto_names += [data_names[i][0:-11]]
    medians += [med]
  # List of medians' percentile values
  scale =list(np.percentile(medians, np.arange(0, 101, perc)))
  # Dictionary of segments, where keys are numbers of segments and values are names of tkens
  # belonging to this segments
  data = {}
  # Loop for sorting cryptos into segments
  for j in range(len(scale)-1):
    temp = []
    for i in range(len(medians)):
      if medians[i] > scale[j] and medians[i] <= scale[j+1]:
        temp += [crypto_names[i]]
    data[j] = temp
  data[0] +=min(crypto_names[np.argmin(medians)])
  N = len(crypto_names)
  A = np.zeros((N,N))
  # Filling of the incidence matrix
  for i in range(N):
    for j in range(N):
      for k in range(len(data.keys())):
        if crypto_names[i]!=crypto_names[j]:
          if crypto_names[i] in data[k] and crypto_names[j] in data[k]:
            A[i][j]=1
  return A

In [0]:
vol_data = np.load('data/Vol_file.npy')
data_names = ['LINK_hourly.csv',
 'HT_hourly.csv',
 'OMG_hourly.csv',
 'VEN_hourly.csv',
 'MTL_hourly.csv',
 'KNC_hourly.csv',
 'REP_hourly.csv',
 'IOST_hourly.csv',
 'BTM_hourly.csv',
 'ZRX_hourly.csv',
 'AE_hourly.csv',
 'NULS_hourly.csv',
 'ELF_hourly.csv',
 'ZIL_hourly.csv',
 'KCS_hourly.csv',
 'WTC_hourly.csv',
 'BNT_hourly.csv',
 'ITC_hourly.csv',
 'NAS_hourly.csv',
 'CVC_hourly.csv',
 'ABT_hourly.csv',
 'THETA_hourly.csv',
 'HPB_hourly.csv',
 'LUN_hourly.csv',
 'ARN_hourly.csv']

In [0]:
# Get 24 month tensor of graphs
A_volume_tensor = []
for i in range(1,25):
  t = i*round(vol_data.shape[0]/24)
  n = round(vol_data.shape[0]/24)
  A_volume = get_At_matrix(vol_data, data_names, t, n, perc=10)
  A_volume_tensor += [A_volume.reshape(A_volume.shape[0], A_volume.shape[1], 1)]
A_volume_tensor = np.concatenate(tuple(A_volume_tensor), axis = 2)